# Falabella

In [2]:
import pandas as pd
df_ml = pd.read_csv("../data/processed/FalabellaAyuda_2022-12-27.csv")
df_ml.dropna(subset=["tweet_text"], inplace=True)
df_ml = df_ml[df_ml["sentiment"] != "others"]
df_ml = df_ml.sample(n=100)
# df_r = pd.read_csv("../data/processed/RipleyChile_2022-12-27.csv")
# df_p = pd.read_csv("../data/processed/tiendas_paris_2022-12-27.csv")

In [3]:
def df_to_list(df):
    texts = df["tweet_text"]
    data = []
    for text in texts:
        data.append(text)
    return data

In [4]:
import numpy as np
import torch
from transformers import BertModel, BertTokenizer
tokenizer_español = BertTokenizer.from_pretrained("../pytorch/", do_lower_case=False)
model = BertModel.from_pretrained("../pytorch")

Some weights of the model checkpoint at ../pytorch were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
from src import Features
inputs = df_to_list(df_ml)
inputs = Features.bert_encoder(inputs, tokenizer_español)
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
data_ml = {
        "max_polling_std": Features.std_normalization(Features.max_polling(last_hidden_states)),
        "max_polling_z_score": Features.z_score_normalization(Features.max_polling(last_hidden_states)),
        "max_polling_max": Features.min_max_normalization(Features.max_polling(last_hidden_states)),
        "mean_polling_std": Features.std_normalization(Features.mean_polling(last_hidden_states)),
        "mean_polling_z_score": Features.z_score_normalization(Features.mean_polling(last_hidden_states)),
        "mean_polling_max": Features.min_max_normalization(Features.mean_polling(last_hidden_states)),
    }

In [6]:
def find_indexs(list, value):
    list_aux = list.copy()
    output = []
    while True:
        try:
            index = list_aux.index(value)
            output.append(index)
            list_aux[index] = None
        except ValueError:
            break
    return output

# agrupamiento

# k-means

In [7]:
# k-means
from sklearn.cluster import KMeans
from sklearn import metrics
agrupamiento = {}
metric = []
for i in range(100):
    metrica = []
    for key, value in data_ml.items():
        kmeans = KMeans(
            n_clusters=4,
            init="random",
            max_iter=100,
            random_state=i,
            n_init=1,
        ).fit(value)
        agrupamiento[key] = list(kmeans.labels_)
        metrica.append(metrics.silhouette_score(
            value, kmeans.labels_, metric="sqeuclidean"))
    metric.append(metrica)
    #print(f"{round(metrica[0], 3)} {round(metrica[1], 3)} {round(metrica[2], 3)} {round(metrica[3], 3)} {round(metrica[4], 3)} {round(metrica[5], 3)}")
rs = []
for i in range(6):
    list_aux = []
    for metrica in metric:
        list_aux.append(metrica[i])
    max_ = max(list_aux)
    index = list_aux.index(max_)
    rs.append([index, round(max_, 3)])
rs

[[37, 0.546], [24, 0.176], [59, 0.493], [67, 0.236], [37, 0.269], [41, 0.254]]

In [8]:
i=0
for key, value in data_ml.items():
    kmeans = KMeans(
        n_clusters=4,
        init="random",
        max_iter=100,
        random_state=rs[i][0],
        n_init=1,
    ).fit(value)
    agrupamiento[key] = list(kmeans.labels_)
    i += 1
clusters = {}
for key, value in agrupamiento.items():
    clusters[key] = {
    "cluster0": df_ml.iloc[find_indexs(agrupamiento[key], 0)],
    "cluster1": df_ml.iloc[find_indexs(agrupamiento[key], 1)],
    "cluster2": df_ml.iloc[find_indexs(agrupamiento[key], 2)],
    "cluster3": df_ml.iloc[find_indexs(agrupamiento[key], 3)],
}
# preprocesado vs sentimiento.
print("rs rsScore cluster 1 cluster 2 cluster 3 cluster 4")
i=0
for key0, value0 in clusters.items():
    data = []
    for key1, value1 in value0.items():
        sentiment = dict(value1["sentiment"].value_counts())
        sentiment_values = list(sentiment.values())
        sentiment_labels = list(sentiment.keys())
        sentiment = [f"{label} ({value})" for label, value in zip(sentiment_labels, sentiment_values)]
        data.append(sentiment)
    string = f"{rs[i][0]} & {rs[i][1]} & {data[0]} & {data[1]} & {data[2]} & {data[3]}"
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace("'", "")
    print(string)
    i += 1

rs rsScore cluster 1 cluster 2 cluster 3 cluster 4
37 & 0.546 & anger (2), sadness (1) & anger (2), joy (2) & anger (2), joy (1) & anger (71), sadness (16), joy (3)
24 & 0.176 & anger (65), sadness (14), joy (2) & anger (2), joy (1) & anger (7), joy (3), sadness (2) & anger (3), sadness (1)
59 & 0.493 & anger (27), sadness (3) & joy (1) & anger (12), sadness (4), joy (3) & anger (38), sadness (10), joy (2)
67 & 0.236 & anger (12) & anger (56), sadness (14), joy (2) & anger (6), joy (2) & anger (3), sadness (3), joy (2)
37 & 0.269 & anger (3), sadness (1) & anger (6), joy (1) & anger (7), sadness (4), joy (3) & anger (61), sadness (12), joy (2)
41 & 0.254 & anger (3), joy (1), sadness (1) & anger (26), sadness (7), joy (4) & anger (3), joy (1) & anger (45), sadness (9)


# Extracción de infrmación

## Modelo TCRE 

In [9]:
# Example usage
corpus = []
key_list = list(clusters.keys())
for _, value0 in clusters[key_list[0]].items():
    corpus.append(value0['tweet_text'].tolist())
corpus[0]

['no les contestes por privado', 'dejen de acosarme', 'sigo sin respuesta']

In [10]:
grupo = 0
corpus0 = []
cluster_result = []
for text_list in corpus:
    for text in text_list:
        corpus0.append(text)
        cluster_result.append(grupo)
    grupo += 1

In [14]:
from src import Summarize
results = Summarize.TCRE(corpus0, cluster_result, 1)
print(results)

[['dejen', 'contestes', 'acosarme', 'privado', 'sigo', 'respuesta', 'servicio', 'pedido', 'rt', 'navidad', 'falabella', 'mas', 'regalo', 'ayuda', 'esperando', 'compra', 'hacen', 'mierda', 'van', 'aun', 'si', 'igual', 'problema', 'solo', 'dias', 'ustedes', 'producto', 'gracias', 'jojojo', 'dinero', 'cliente', 'muchas', 'dia', 'verguenza', 'estan', 'compre', 'ser', 'tiempo', 'acabo', 'hola', 'vez', 'excelente', 'dejaron', 'feliz', 'necesito', 'solucion', 'responder', 'espero', 'peor', 'responden', 'llegara', 'pesimo', 'devolucion', 'quiero', 'sinverguenzas', 'empresa', 'horrible', 'atencion', 'entregar', 'demoran', 'llego', 'mensaje', 'nadie', 'puras', 'tan', 'estafadores', 'dan', 'seria', 'hija', 'dicen', 'da', 'reembolso', 'datos', 'caso', 'algun', 'mismo', 'ojala', 'senores', 'siempre', 'reclamo', 'llega', 'puede', 'hice', 'ahora', 'cada', 'fecha', 'despacho', 'desean', 'arruinaron', 'ninos', 'corazon', 'solucionar', 'medio', 'devuelvan', 'alguna', 'tres', 'mal', 'atienden', 'compren'